<a href="https://colab.research.google.com/github/rooncal/Image-Segmentation/blob/main/Training_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports

In [ ]:
import torch
from tqdm import tqdm
import sys
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 155kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


##Cloning model from GitHub

In [ ]:
!git clone https://github.com/jacobkimmel/fcdensenet_pytorch.git

Cloning into 'fcdensenet_pytorch'...
remote: Enumerating objects: 90, done.
remote: Total 90 (delta 0), reused 0 (delta 0), pack-reused 90
Unpacking objects: 100% (90/90), done.


##Importing related classes

In [ ]:
sys.path.append("/content/fcdensenet_pytorch/")
from fcdensenet_pytorch.model import DenseNet103
from fcdensenet_pytorch.trainer import DiceLoss
from fcdensenet_pytorch.data_loader import CellDataset

ImportError: ignored

##Getting data

This line downloads and decompresses the dataset to a folder named "data"

In [ ]:
!if [[ ! -e data ]]; then wget http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamSeq01/CamSeq01.zip; unzip -qq CamSeq01.zip -d data; rm -f CamSeq01.zip; fi

--2021-05-08 10:33:38--  http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamSeq01/CamSeq01.zip
Resolving mi.eng.cam.ac.uk (mi.eng.cam.ac.uk)... 129.169.82.147
Connecting to mi.eng.cam.ac.uk (mi.eng.cam.ac.uk)|129.169.82.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94535380 (90M) [application/zip]
Saving to: ‘CamSeq01.zip’

CamSeq01.zip        100%[===================>]  90.16M  21.6MB/s    in 9.0s    

2021-05-08 10:33:47 (10.0 MB/s) - ‘CamSeq01.zip’ saved [94535380/94535380]



In [ ]:
!rm -f data/*.txt
!mkdir labels
!mv data/*L.png labels

mkdir: cannot create directory ‘labels’: File exists
mv: cannot stat 'data/*L.png': No such file or directory


In [ ]:
train_set = CellDataset('/content/data', '/content/labels')

NameError: ignored

##Our training class

In [ ]:
class Trainer():
  def __init__(self, model, loss_function, optimizer, train_dataloader, start_epoch=0, val_dataloader=None, tqdm=True,
               tensorboard_log_dir='', lr_scheduler=None, model_checkpoint_dir='', experiment_name='model_training'):
    self.model = model
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.loss_function = loss_function
    self.epoch = start_epoch
    self.optimizer = optimizer
    self.train_dataloader = train_dataloader
    self.val_dataloader = val_dataloader
    self.tensorboard_log_dir = ''
    self.lr_scheduler = lr_scheduler
    self.model_checkpoint_dir = model_checkpoint_dir
    self.tqdm = tqdm
    self.experiment_name = experiment_name


    def train(self, total_epochs, checkpoint_interval=10):
      for self.epoch in range(self.epoch, total_epochs):
        self.train_iter()
        if self.val_dataloader:
          self.val_iter()
        if checkpoint_interval % epoch == 0:
          save_checkpoint()


    def train_iter(self):
      running_loss = 0
      self.model.train()

      if self.tqdm:
        self.train_dataloader = tqdm(self.train_dataloader)

      for (image, mask) in self.train_dataloader:
        self.optimizer.zero_grad()
        image, mask = image.to(self.device), mask.to(self.device, dtype=torch.long).squeeze(1)
        output = self.model(image)
        loss = self.loss_function(output, mask)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
      
      print(f"Epoch: {self.epoch}, Training Loss: {running_loss}")
      return running_loss

    def val_iter(self):
      self.model.eval()
      if self.tqdm:
        self.val_dataloader = tqdm(self.val_dataloader)

      with torch.no_grad():
        running_loss = 0
        for (image, mask) in self.val_dataloader:
          image, mask = image.to(self.device), mask.to(self.device,dtype=torch.long).squeeze(1)
          output = self.model(mask)
          loss = self.loss_function(output, mask)
          running_loss += loss.item()

        print(f"Epoch: {self.epoch}, Validation Loss: {running_loss}")
        return running_loss
    
    def save_checkpoint(self):
      torch.save({
            'epoch': self.epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            }, os.path.join(model_checkpoint_dir,self.experiment_name,f"checkpoint_{self.epoch}")) 
        
    def load_checkpoint(self, path, epoch=0):
      if epoch and not path:
        path = os.path.join(model_checkpoint_dir,self.experiment_name,f"checkpoint_{epoch}")
      checkpoint = torch.load(path)
      self.model.load_state_dict(checkpoint['model_state_dict'])
      self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      self.epoch = checkpoint['epoch']
      


##Training the model

In [ ]:
densenet = DenseNet103()
optimizer = torch.optim.RMSprop(densenet.parameters())
densenet_trainer = Trainer(model=DenseNet103,loss_function=DiceLoss,optimizer=optimizer,)